# Attention 前世今生

# 目录

- [参考文献](#参考文献)
- [Attention作用](#Attention作用)
- [Attention的提出](#Attention的提出)
    - [背景](#背景)
    - [Attention_in_Seq2Seq](#Attention_in_Seq2Seq)
- [Attention提出的意义](#Attention提出的意义)
- [有趣的联想](#有趣的联想)
- [通用定义](#通用定义)
- [计算步骤](#计算步骤)
- [Attention的多种形式](#Attention的多种形式)
    - [Local_Attention](#Local_Attention)
    - [Self-Attention](#Self-Attention)
    - [Key-Value_Attention](#Key-Value_Attention)
    - [Transformer——集Attention大成者](Transformer——集Attention大成者)


# 参考文献

- https://zhuanlan.zhihu.com/p/91315967


# 作用

Attention机制可以让神经网络**更多的关注**到输入中**相关的信息**，并**减少对无关信息的注意**。帮助神经网络**更好的利用输入的信息**，有助于提升模型的可解释性（例如，词语间的相关性可视化）。

## Attention的提出

## 背景

![](../imgs/seq2seq1.jpg)

上图seq2seq模型存在问题：尤其输入长句时，Encoder输出Context向量给Decoder，由于Context长度有限，所以难以充分捕捉输入句子的信息。



## Attention_in_Seq2Seq

![Attention机制在Seq2Seq中应用的示意图](../imgs/attseq2seq.jpg)



上面左图：

Decoder的输入$s_1$（seq2seq中的context）和Encoder中的$h_1,h_2,h_3$进行计算，得到相应的attention scores，然后得到attention distribution。从图中可以看出，$s_1$对“好”字更敏感，分布值也最高，输出的attention output(即context向量)$c_1$中$h_3$的比例最高，对应的由$c_1$拼接$s_1$而成的新向量在Decoder中映射出的单词是“good”。

对比上一张图中seq2seq，attention seq2seq使用$c_1$拼接$s_1$作为context向量，增加了向量的信息量。

上面右图：

左图中（即上一step）decoder输出的结果“good”又作为decoder的输入，得到输入向量$s_2$，输入$s_2$和Encoder中的$h_1,h_2,h_3$进行计算，分别得到attention scores$e_1,e_2,e_3$，在经过$softmax$计算得到attention distribution，分布值分别为$α_1,α_2,α_3$。$α_1,α_2,α_3$再分别与$h_1,h_2,h_3$相乘并sum结果，得到attention outpu$c_2$.可以看到“早上”二字的分布值较大，在$s_2$对应的context向量中占较大比例，对应step输出为"morning"。

继续如此到“enb”为止。

attention output（context向量）计算公式如下：

$$c_i = \sum_{j=1}^{T_x}α_{ij}h_j$$

其中$i$表示decoder的第i个输入，$j$表示在encoder输出的第$j$个隐藏变量。$c_i$是decoder的第$i$个输入对应的context向量（decoder第i个输入与encoder每个时间步隐层向量的加权和，对应的权重为$α_{ij}$），$h_j$是encoder输出的第$j$个隐藏层变量，$α_{ij}$是decoder第$i$个输入与encoder第$j$个隐藏层变量对应的权重。

注意力权重(注意力分布值)计算公式：

$$α_{ij}=\frac{exp(e_{ij})}{\sum_{k=1}^{T_x}exp(e_{ik})}$$

注意力分数$e_{ij}$表示在decoder运行到第$i$时部时，对应于encoder第$j$个序列隐藏层输出的注意力分数计算公式如下：

$$e_{ij}=v_a^Ttanh(W_as_i + U_ah_j)$$

其中$s_i$为decoder第$i$时部的隐层向量，$h_j$是encoder的第$j$个隐层向量。$v_a$是一个向量，用于将向量转化为实数。

实例如下：

![ed例子](../imgs/rnned1.jpg)

# Attention提出的意义

- Attention机制解决了定长context向量导致的信息瓶颈问题
- 缓解RNN（包括LSTM）在长距离依赖中梯度消失的问题（Decoder可以自由的对RNN的各个时步直接建立连接）
- 提供了一定的可解释性（论文中给出的对齐矩阵如下图）。

![对齐矩阵](../imgs/attmat.jpg)

# 有趣的联想

在LSTM和GRU， 以及ResNet中，其实已经蕴藏着attention的思想。用**加法(sum)**的形式将**参数(weight)**以**不同的权重(score)**组合起来，就是attention的精髓了。

只不过这些模型中权重是由各种门决定(Gate)，注意到门的输出也是**(0, 1)区间**，和Attention权重由Softmax输出在(0, 1)之间也保持一致。

且他们（Gate与Attention层）均为joint learning，即通过和目标一起联合学习得到。

# 通用定义

**给出一组值向量(values)和一个查询向量(query)，attention是一种根据查询向量(query)计算这组值向量(values)的加权和的方法。**

例如在Seq2Seq中，encoder各个时步的隐层向量$h_j$组成了我们的值向量(values)，而decoder的在第i步输入时的隐层向量 $s_i$ 是我们的查询向量(query)。我们即是根据这个 $s_i$ 计算出我们全体 $h_j$ 的加权和，这个加权和即为我们的Attention输出。

更进一步，这个**加权和就是对values中的信息的一个有选择性的概要，我们的query决定了values中的哪一部分会被关注，会被着重提取**。

同时Attention也是一种获取一组长度不定的向量(values)依赖于另一个向量(query)的定长表示的方式。

# 计算步骤

假设有一组向量(values)$h_1,h_2,h_3,..,h_N \in R^{d_h}$以及一个查询向量(query)$s \in R^{d_s}$,计算attention需要以下三个步骤：

1. 计算attention scores $e \in R^N$  (这一步的实现方法很多)
2. 通过softmax获得权重 $α=softmax(e) \in R^N$
3. 根据得到对权重分布计算加权和(context向量) $c=\sum_{i=1}^{N}α_ih_i \in R^{d_h}$


【相关计算注意力得分公式未列出，请在原文中找 https://zhuanlan.zhihu.com/p/91315967 】

# Attention的多种形式

## Self-Attention

在文本分类，文本推荐等领域（N to 1），虽然输入是一个序列（或者说一组向量），但是输出却不是（输出通常为一个值）。我们有values，但是似乎难以寻找一个额外的query。例如文本分类中，除了文本本身并没有其他输入。

针对这样的场景， Yang et al.在2016年提出了Self Attention [ Yang et al. 2016]。 顾名思义，**self attention的query和values都属于同一个序列**。

观察到**对于一个句子的向量表示，各个词在其中的贡献程度都是不一样的。同样对于一篇文章的向量表示而言，各个句子对其的贡献程度也是不同的。**Yang et al.希望**通过Self attention机制可以帮助提取出相对更重要的词语或句子**。

给出句子层级的Context 向量 $u_s$ ，第i个句子的RNN隐层向量为$h_i$,求文章的表示向量$v$,有

$u_i = tanh(W_sh_i+b_s)$ attention score

$$α_i = \frac{exp(u_i^Tu_s)}{\sum_texp(u_i^Tu_s)}$$

$$v = \sum_{t}α_ih_i$$

这里 $u_s$可以理解为对于“哪一句话携带最多能帮助文章分类信息”这个问题(或者说query)的一种向量表示，可以一开始对 $u_s$进行随机初始化，然后再进行joint learning, 这里可以参考Sukhbaatar et al.对meomory network中问题文本的向量化处理 [Sukhbaatar, et al. 2015].

## Key-Value_Attention

之前见到的Attention中，Values同时兼具了多个作用，例如在LM(Language Model)中，既包含了词语本身的encoding（$h_j$）用于预测下一个词语，又要用于attention计算以提供Context向量。

Daniluk et al.认为这样的重用可能会使得训练困难，因此他们在2017年提出了Key-value Attention [Daniluk, Michał, et al. 2017]，**Key用于计算attention，Values用于词语本身的encoding**。

![](../imgs/kvatt.jpg)

$[k_t,v_t]=h_t \in R^{2k}$

$M_t=tanh(W^Y[k_{t-L}...k_{t-1}]+(W^hk_t)1^T) \in R^{k*L}$

$α_t=softmax(\omega ^T M_t) \in R^{1*L}$

$r^t=[v_{t-L}...v_{t-1}]α^T \in R^k$

$h_t$是t时刻的encoder的隐层输出，分为了$k_t,v_t$两个输出，二者分别对应t时刻的key向量和value向量。

$M_t$是t时刻的attention score，采用可加形式(addictive)，其中$L$是Attention的窗口长度，这里取的是当前时刻L个时步(论文应用于LM，无法像NMT领域一样取前后时步作为窗口). $r_t$ 是t时刻的对应Context向量。

## Transformer——集Attention大成者

可变长度数据的表示学习应用方向：

- 机器翻译
- 文本摘要
- QA等

### Scaled Dot-Prodcut Attention

Transformer同样采用了Key-Value Attention的设计，不过出于性能考虑，它使用的不是addictive的形式，而是dot product形式(见softmax里面的计算):

$Attention(Q,K,V)=softmax(\frac{QK^T}{\sqrt{d_k}})V$

其中$Q \in R^{|Q|*d_k}$是query向量构成的矩阵，$K \in R^{|K|*d_k}$是key向量构成的矩阵，$V \in R^{|K|*d_v}$是values向量构成的矩阵。$|Q|$是query的数量，$|K|$是key的数量也是key-value对中的value的数量。分母$\sqrt{d_k}$是一个数值上的处理，为避免dot product过大引入的缩放项，$d_k$是key向量维度，也是query向量的维度。

虽然Transformer也是采用的Self Attention，但是这里的Self Attention不同于我们在之前的 [Yang, Zichao, et al 2016] 中看到的Self Attention。之前我们看到的Self Attention可以理解为把每一个query都作为一个同参数向量来joint learning的，而这里采用的query向量就是这个词语的词向量（在后面提到的Multi-head版本中是词向量的降维形式）。

他们的keys和values都是一致的，均为句子中各个词语的词向量。

### Multi-Head Attention

单层的Attention涵盖的信息可能不足以支持多种下游任务，Transformer在前面Attention的设计之上继续叠加成为了Multi-Head Attention。

$$MultiHead(Q,K,V)=Concat(head_1,....,head_h)W^O$$

$$head_i=Attention(QW^Q_i,KW_i^K,VW_i^V)$$

其中$W^Q_i \in R^{d_{model}*d_k},W^K_i \in R^{d_{model}*d_k},W^V_i \in R^{d_{model}*d_v},W^O \in R^{hd_{v}*d_{model}}$,$d_{model}$是模型中q，k，v向量的维度，在Transformer模型中输入的单条query，key，value向量的维度都是一样的，这里用$d_{model}$表示。

$h$是multi-head的head数，即**并行**输出的attention层的层数，是一个超参，模型中取8.

$d_k,d_v$是将高维的$d_{model}$维度通过相应的$W$矩阵映射后的维度，在模型中取$d_k=d_v=\frac{d_{model}}{h}=64$

下面我们结合上式对计算MultiHead Attention的过程进行一个描述。

我们首先对输入的高维向量 $Q,K,V$ 通过 $W$ 矩阵进行降维，然后进行Scaled Dot-Product Attention得到我们的Attention输出，再将输出拼接在一起，通过 $W^O$ 矩阵还原为高维形式。

通过引入Multi-Head和相应的降维操作，我们的计算量实际上和单一层数近似，同时又获得了更丰富的表征。

### Attention的因子分解

X Zhu et al. 在2019年提出Transformer中的Attention机制可以分解为4个因素 [X Zhu et al. 2019] (该论文目前仍是preprint版本，尚未发表到会议)

![](../imgs/trans1.jpg)

可以看出(a)(b)因素是和query内容相关的，(c)(d)因素是和query内容无关的。

通过实验，他们的结论是：

在Self Attention中，仅使用与query内容无关的(c)(d)因素就可以可以达到完全使用4个因素效果的大部分作用，仅使用(b)(c)因素就可以达到非常接近于使用全部4个因素的效果，而(a)因子对结果影响很小，这个因子恰恰是计算量最大的地方。

但是在Encoder-Decoder Attention中，仅使用(a)因子就可以很接近使用全部4个因子的效果，4个因子中(a)因子是最关键的。

因此，在**Attention计算量过大的图像等领域，我们或许可以优化掉一些因子来加速计算**。


# 总结

Attention机制从**解决Seq2Seq中信息瓶颈问题**而来，用**加权和**的形式聚合向量，简单而有效。当我们在模型中具体运用它时，我总结了以下几点可以提供一个设计和思考的方向：

1. 确定场景中的query和values（或者query, key-values）
2. 确定Attention Score计算方法
3. 根据问题考虑是否采用更特殊的Attention设计
4. 优化Attention性能


下面针对这几点进行一个更详细的说明。


- **确定场景中的query和values** 例如在Seq2Seq中，query是decoder中当前待翻译的词语，values是encoder中的各个词语的隐向量。在Self-Attention中，query可以用一个待学习的向量代替，也可以用当前词向量（或其映射）做query，values则是整个句子的各个词向量。在VQA中，问题文本的一个词是query，图像的feature map是values，同时图像的一个区域是query，问题文本的各个词语是values。其他领域的query和values，以及query, key-values也可以类似的确定。


- **确定Attention Score计算方法** 最常用的点乘(Dot product)和加法(Additive)，以及其他各个变种(如乘法(Multiplicative)，拼接(Concat))。可以从query和values(keys)的维度，计算效率，训练难度，还有问题本身的结构和性质等方面考虑。如果query和values(keys)维度一样，那么可以直接点乘，不一样的话需要考虑用乘法形式，不一样时也可以用拼接或者加法形式。点乘的计算效率比加法高，但是也要注意数值上的突变问题，考虑像Transformer一样加入scala项缓和该问题。如果问题本身有特殊的结构，相似性度量方法，需要根据具体情况选用相应的形式。


- **更特殊的Attention设计**
    - **multi-head设计** 如果有**多个下游任务**，或者**目标任务需要关注的信息较多**，那么可以采取多层attention设计。是否需要像Transformer一样对各层输入进行降维可以根据计算量和模型结构决定。是否需要像Lin et al.那样在多维模型 [Lin et al. 2017] 中那样加入惩罚项，可以对attention权重进行可视化后，判断是否有效的学习到了多种信息后再决定。
    - **层次设计** 这个由问题特性决定。例如文章-句子-词语，句子-词语-字等天然具有层次特性。事实上多层次对模型结构本身影响不大，主要是需要避免训练过于困难。
    - **Co-Attention设计** 这个也由问题特性决定，问题中如果query和values具有对称性可以考虑。


- **优化Attention性能** 在大规模的应用中，常常会遇到性能瓶颈，这时候可以考虑对Attention性能进行进一步的优化。
    - **确定attention的范围** 是local attention还是global attention，是固定窗口还是通过学习参数矩阵来预测，窗口的方向是双向还是单向
    - **Hard Attention** Hard Attention带来了一些训练和设计的困难，但是也减少了计算量
    - **因子分解** 可以考虑分解成多个因子，保留最关键的计算部分，如在特定cv领域可以删掉计算query和values内容相关性的部分